# Decision Tree

### Load libraries

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn import datasets
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import math 

C:\ProgramData\Anaconda\lib\site-packages\statsmodels\compat\pandas.py:23: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


### Loading Data

**Lets read the example dataset which contains the GRE score, GPA and rank of students and whether they are accepted or not at the university (admit column)**

In [18]:
students = pd.read_csv("https://stats.idre.ucla.edu/stat/data/binary.csv")

**we move the admit to the last column in the dataframe**

In [19]:
df0 = students.pop('admit')
students['admit']=df0

## Decision Tree

**For writing a decision tree we need some functions. The first function (left_right) devides the dataset into left and right data
according to the split criteria (splt_crt) and the column number of the data (cl_num) which we want to split.**

In [20]:
def left_right(data,cl_num,splt_crt):
    Y = data.iloc[:,-1]
    X = data.iloc[:,0:len(data)-1]
    left = X[X.iloc[:,cl_num] < splt_crt]
    right = X[X.iloc[:,cl_num] >= splt_crt]
    return left, right

**We also need to define a split criteria and select what feature should split first in our tree. `Info_gain` function returns column number (col_min)
of the feature that should be splitted first and the split criteria for that feature according to the Gini Index. It also returns 
the Impurity weight (weight_Imp) which helps us to decide after what depth we do not need more split and spliting does not add too much to our information (one of the tresholds to terminate the devision).**

In [21]:
def Info_gain(data):
    gini = np.zeros((len(data),len(data.columns)-1))
    w_Imp = np.zeros((len(data),len(data.columns)-1))
    for i in range(len(data.columns)-1):
        for j in range(len(data)):
            splt_crt = data.iloc[j,i]
            left, right = left_right(data,i,splt_crt)
            Y_l = left.iloc[:,-1]
            Y_r = right.iloc[:,-1]
            if len(Y_l)== 0:
                left_Imp = 0
            else:
                left_Imp = 1-(sum(Y_l)/len(Y_l))**2-((len(Y_l)-sum(Y_l))/len(Y_l))**2   ## left Impurity
            if len(Y_r)== 0:
                right_Imp = 0
            else:
                right_Imp = 1-(sum(Y_r)/len(Y_r))**2-((len(Y_r)-sum(Y_r))/len(Y_r))**2  ## right Impurity
            Imp = 1-(len(Y_l)/len(data))**2-(len(Y_r)/len(data))**2                     ## Impurity
            gain = Imp-(len(Y_l)/len(data))*left_Imp-(len(Y_r)/len(data))*right_Imp     ## Information Gain
            w_Imp[j,i] = (len(Y_l)+len(Y_r))/len(data)*gain                             ## Impurity Weight
            gini[j,i] = (len(Y_l)/len(data))*left_Imp+(len(Y_r)/len(data))*right_Imp    ## Gini Index
    row_min = np.argmin(np.min(gini, axis=1))
    col_min = np.argmin(np.min(gini, axis=0))
    splt_crt = data.iloc[row_min,col_min]
    weight_Imp = w_Imp[row_min,col_min]
    return col_min, splt_crt, weight_Imp

**As we need the output of the decision tree, we define a function that terminates the tree branches and print the admit
result (0 or 1)**

In [22]:
def terminate(group):
    max_value_count = list(group.iloc[:,-1].mode())
    return max_value_count

**Split function is the main function of the decision tree algorithm. We recursively split the dataset to right and left branches according to the split criteria (Gini Index or Entropy). This function needs the dataset that we want to create a decision tree for it `(features)`, the minimum row numbers to define the minimum required observation in each branch that we are allowded to split `(min_rows)`, maximum depth of the tree `(max_depth)`, a threshold `(tresh)` for Impurity weight (If the Impurity weight is less than threshold we terminate the tree because spliting does not add any information for us), `depth` which is the numerator for how deep we go in the tree and the `result` which the function returns(contains data column index(the data that is splitted), split criteria, depth of the current branch, the branch name (left or right). For the end of a tree branch we print -1 and -1 which means None value, depth, left or right branch and admit value (1 or 0))**

In [23]:
def split(features,min_rows,max_depth,tresh,depth,result):
    cl_num,splt_crt,weight_Imp= Info_gain(features)
    left,right = left_right(features,cl_num,splt_crt)
    if depth >= max_depth:
        result.append([-1,-1,depth,'left',np.array(terminate(left))[0]])
        result.append([-1,-1,depth,'right',np.array(terminate(right))[0]])
        return
    if len(left)<= min_rows or weight_Imp <= tresh:
        result.append([-1,-1,depth,'left',np.array(terminate(left))[0]])
    else:
        result.append([Info_gain(left)[0],Info_gain(left)[1],depth,'left'])
        split(left,min_rows,max_depth,tresh,depth+1,result)
    if len(right) <= min_rows or weight_Imp <= tresh:
        result.append([-1,-1,depth,'right',np.array(terminate(right))[0]])
    else:
        result.append([Info_gain(right)[0],Info_gain(right)[1],depth,'right'])
        split(right,min_rows,max_depth,tresh,depth+1,result)            
    return result

**To predict the admit results for test data we need to define a predict function for our decision tree. I saved the decision tree in the `split` function as a list of branches that each row of the list is a branch (Materialized Path format). In the predict function I first read the first row of the list which is the first node with depth=0 and compare the test data with its criteria, If the test data feature value is less than that criteria we need to move to the left branch with depth 1, In that case we do not need all branches that are made by right branch with depth=1. This helps to get rid of unneccessary branches, in the next step we compare the data value to the second node condition (second on the left here) and if it is greater than the criteria for example we need to move to the depth 2 (depth+1) and left branch. Then we remove all branches that are made with the left node at depth=2. This approach continues untill we reach to the end of the decision tree and finally return the prediction value of that node.**

In [24]:
def predict(data,tree,row):
    depth = 0
    n = len(tree)-1
    d = max(tree['Depth'])
    row_b=0
    while (tree['Index'][row_b])>=0:
        if data.loc[row][int(tree['Index'][row_b])] < tree['Criteria'][row_b]:
            depth = depth+1
            row_b = int(tree[(tree['Depth']==depth) & (tree['branch']=='left')].index.tolist()[0])
            row_end =int(tree[(tree['Depth']==depth) & (tree['branch']=='right')].index.tolist()[0])
            tree = tree.loc[row_b:row_end,:]
            prd = tree['value'][row_b]
            
        else:
            depth = depth+1
            row_b = int(tree[(tree['Depth']==depth) & (tree['branch']=='right')].index.tolist()[0])
            tree = tree.loc[row_b:,:]
            prd = tree['value'][row_b]
            
    return prd

**Lets devide our dataset to test and train group**

In [25]:
test = students.sample(frac=0.2,random_state=4)
train_mask = pd.Series(True, index=students.index)
train_mask[test.index] = False
train = students[train_mask].reset_index()
del train['index']
test = test.reset_index()
del test['index']

### 1. Decision tree result With Gini Index

**Lets create a decision tree with the train dataset with maximum depth = 5**

In [26]:
res = [[Info_gain(train)[0],Info_gain(train)[1],0,'left']]
tree = split(train,10,5,0,1,res)
df = pd.DataFrame(tree,columns=['Index','Criteria','Depth','branch','value'])

**Lets predict the train admit values with our decision tree (we want this to check for overfitting by comparing with the test prediction)** 

In [27]:
prd_train = np.zeros((len(train),3))
for row in range(len(train)):
    prd_train[row,0] = predict(train,df,row)
    prd_train[row,1] = train['admit'][row]
    prd_train[row,2] = 1

**Lets create the confusion matrix**

In [28]:
prd_admt_train = pd.DataFrame(prd_train,columns=['Estimate','admit','count'])
out_counts_train = prd_admt_train.groupby(['Estimate', 'admit'])['count'].count()
out_counts_train.unstack()

admit,0.0,1.0
Estimate,,
0.0,210,77
1.0,7,26


**Lets see the accuracy of the decision tree for train data**

In [29]:
(out_counts_train[0][0]+out_counts_train[1][1])/sum(out_counts_train)

0.7375

**Now, we predict the admit values with test dataset**

In [30]:
prd_test = np.zeros((len(test),3))
for row in range(len(test)):
    prd_test[row,0] = predict(test,df,row)
    prd_test[row,1] = test['admit'][row]
    prd_test[row,2] = 1

**Confusion Matrix for test dataset**

In [31]:
prd_admt_test = pd.DataFrame(prd_test,columns=['estimate','admit','count'])
out_counts_test = prd_admt_test.groupby(['estimate', 'admit'])['count'].count()
out_counts_test.unstack()

admit,0.0,1.0
estimate,,
0.0,52,19
1.0,4,5


**The accuracy for the test dataset**

In [32]:
(out_counts_test[0][0]+out_counts_test[1][1])/sum(out_counts_test)

0.7125

**It seems that the model does a little bit overfit and it returns high accuracy**

### 2. Decision tree result With Entropy

**Lets define Info_gain function for entropy**

In [33]:
def Info_gain_entropy(data):
    entropy = np.zeros((len(data),len(data.columns)-1))
    for i in range(len(data.columns)-1):
        for j in range(len(data)):
            splt_crt = data.iloc[j,i]
            left, right = left_right(data,i,splt_crt)
            Y_l = left.iloc[:,-1]
            Y_r = right.iloc[:,-1]
            if len(Y_l)== 0:
                left_Imp = 0
            else:
                p_l = (sum(Y_l)/len(Y_l))
                if p_l==0 or p_l==1:
                    a=0
                    b=0
                else:
                    a = np.log2(p_l)
                    b = np.log2(1-p_l)
                left_Imp = -float(p_l)*float(a)-(1-float(p_l))*float(b)                   ## left Impurity
            if len(Y_r)== 0:
                right_Imp = 0
            else:
                p_r = sum(Y_r)/len(Y_r)
                if p_r==0 or p_r==1:
                    a=0
                    b=0
                else:
                    a = np.log2(p_r)
                    b = np.log2(1-p_r)
                right_Imp = -p_r*a-(1-p_r)*b                                              ## right Impurity
            entropy[j,i] = (len(Y_l)/len(data))*left_Imp+(len(Y_r)/len(data))*right_Imp   ## Entropy
    row_min = np.argmin(np.min(entropy, axis=1))
    col_min = np.argmin(np.min(entropy, axis=0))
    splt_crt = data.iloc[row_min,col_min]
    return col_min, splt_crt

**Split function for entropy**

In [34]:
def split_entropy(features,min_rows,max_depth,depth,result):
    cl_num,splt_crt= Info_gain_entropy(features)
    left,right = left_right(features,cl_num,splt_crt)
    if depth >= max_depth:
        result.append([-1,-1,depth,'left',np.array(terminate(left))[0]])
        result.append([-1,-1,depth,'right',np.array(terminate(right))[0]])
        return
    if len(left)<= min_rows:
        result.append([-1,-1,depth,'left',np.array(terminate(left))[0]])
    else:
        result.append([Info_gain_entropy(left)[0],Info_gain_entropy(left)[1],depth,'left'])
        split_entropy(left,min_rows,max_depth,depth+1,result)
    if len(right) <= min_rows:
        result.append([-1,-1,depth,'right',np.array(terminate(right))[0]])
    else:
        result.append([Info_gain_entropy(right)[0],Info_gain_entropy(right)[1],depth,'right'])
        split_entropy(right,min_rows,max_depth,depth+1,result)            
    return result

**Decision tree for entropy**

In [35]:
res = [[Info_gain_entropy(train)[0],Info_gain_entropy(train)[1],0,'left']]
tree_entropy = split_entropy(train,10,5,1,res)
df_entropy = pd.DataFrame(tree_entropy,columns=['Index','Criteria','Depth','branch','value'])

**Train prediction for entropy**

In [36]:
prd_train = np.zeros((len(train),3))
for row in range(len(train)):
    prd_train[row,0] = predict(train,df_entropy,row)
    prd_train[row,1] = train['admit'][row]
    prd_train[row,2] = 1

**Confusion Matrix for train entropy**

In [37]:
prd_admt_train = pd.DataFrame(prd_train,columns=['Estimate','admit','count'])
out_counts_train = prd_admt_train.groupby(['Estimate', 'admit'])['count'].count()
out_counts_train.unstack()

admit,0.0,1.0
Estimate,,
0.0,203,59
1.0,14,44


**Train accuracy by entropy**

In [38]:
(out_counts_train[0][0]+out_counts_train[1][1])/sum(out_counts_train)

0.771875

**Test prediction by entropy**

In [39]:
prd_test = np.zeros((len(test),3))
for row in range(len(test)):
    prd_test[row,0] = predict(test,df_entropy,row)
    prd_test[row,1] = test['admit'][row]
    prd_test[row,2] = 1

**Test confusion matrix for entropy**

In [40]:
prd_admt_test = pd.DataFrame(prd_test,columns=['estimate','admit','count'])
out_counts_test = prd_admt_test.groupby(['estimate', 'admit'])['count'].count()
out_counts_test.unstack()

admit,0.0,1.0
estimate,,
0.0,50,19
1.0,6,5


**Test accuracy ny emtropy**

In [41]:
(out_counts_test[0][0]+out_counts_test[1][1])/sum(out_counts_test)

0.6875

**Training accuracy for entropy is higher than gini index but the test accuracy for it is less**